# Restoran Arama Botu


Bu atölye çalışması iki bölümden oluşmaktadır:



*   Bölüm 1: Bot alt yapısının oluşturulması
*   Bölüm 2: Doğal dil anlama yetisini kazandırma





# Bölüm 1: Bot alt yapısının oluşturulması

### Kurulum

**Rasa Core Mainline Kurulumu**: 

`!pip install -U git+https://github.com/rasahq/rasa_core.git`

In [0]:
!pip install rasa_core;

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')


!rm -rf /content/babi_stories.md*
!rm -rf /content/franken_data.json*
!rm -rf /content/wget-log*
!wget https://raw.githubusercontent.com/xentnex/deepcon18_workshop/master/data/babi_stories.md -P /content
!wget https://raw.githubusercontent.com/xentnex/deepcon18_workshop/master/data/franken_data.json -P /content


### Domain Tanımı


In [13]:
domain_yml = """
slots:
  cuisine:
    type: text
  people:
    type: text
  location:
    type: text
  price:
    type: text
  info:
    type: text
  matches:
    type: unfeaturized

entities:
- location
- info
- people
- price
- cuisine

intents:
 - greet
 - affirm
 - deny
 - inform
 - thankyou
 - request_info

templates:
  utter_greet:
    - "hey there!"
  utter_goodbye:
    - "goodbye :("
    - "Bye-bye"
  utter_default:
    - "default message"
  utter_ack_dosearch:
    - "ok let me see what I can find"
  utter_ack_findalternatives:
    - "ok let me see what else there is"
  utter_ack_makereservation:
    - text: "ok making a reservation for restaurant (price={price} cuisine={cuisine}) in location={location} for count={people}? "
      buttons:
        - title: "thank you"
          payload: "thank you"
  utter_ask_cuisine:
    - "what kind of cuisine would you like?"
  utter_ask_howcanhelp:
    - "how can I help you?"
  utter_ask_location:
    - "where?"
  utter_ask_moreupdates: 
    - "if you'd like to modify anything else, please tell me what. This is what I currently have: {location} (price: {price}, cuisine: {cuisine}) for {people} people."
  utter_ask_numpeople:
    - "for how many people?"
  utter_ask_price:
    - text: "in which price range?"
      buttons:
      - title: "cheap"
        payload: "cheap"
      - title: "expensive"
        payload: "expensive"
  utter_on_it:
    - "I'm on it"

actions:
- utter_greet
- utter_goodbye
- utter_default
- utter_ack_dosearch
- utter_ack_findalternatives
- utter_ack_makereservation
- utter_ask_cuisine
- utter_ask_howcanhelp
- utter_ask_location
- utter_ask_moreupdates
- utter_ask_numpeople
- utter_ask_price
- utter_on_it
- action_search_restaurants
- action_suggest
"""

%store domain_yml > restaurant_domain.yml

Writing 'domain_yml' (str) to file 'restaurant_domain.yml'.


### Aksiyonlar



In [0]:
from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet

class RestaurantAPI(object):
    def search(self, info):
        return "papi's pizza place"

class ActionSearchRestaurants(Action):
    def name(self):
        return 'action_search_restaurants'

    def run(self, dispatcher, tracker, domain):
        dispatcher.utter_message("looking for restaurants")
        restaurant_api = RestaurantAPI()
        restaurants = restaurant_api.search(tracker.get_slot("cuisine"))
        return [SlotSet("matches", restaurants)]


class ActionSuggest(Action):
    def name(self):
        return 'action_suggest'

    def run(self, dispatcher, tracker, domain):
        dispatcher.utter_message("here's what I found:")
        dispatcher.utter_message(tracker.get_slot("matches"))
        dispatcher.utter_message("is it ok for you? "
                                 "hint: I'm not going to "
                                 "find anything else :)")
        return []


### Diyalog Modelinin Eğitilmesi

In [0]:
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.agent import Agent


import logging

from rasa_core.policies.keras_policy import KerasPolicy

logger = logging.getLogger(__name__)


class RestaurantPolicy(KerasPolicy):
    def model_architecture(self, input_shape, output_shape):
        """Build a Keras model and return a compiled model."""
        from keras.layers import LSTM, Activation, Masking, Dense
        from keras.models import Sequential

        from keras.models import Sequential
        from keras.layers import \
            Masking, LSTM, Dense, TimeDistributed, Activation

        # Build Model
        model = Sequential()

        # the shape of the y vector of the labels,
        # determines which output from rnn will be used
        # to calculate the loss
        if len(output_shape) == 1:
            # y is (num examples, num features) so
            # only the last output from the rnn is used to
            # calculate the loss
            model.add(Masking(mask_value=-1, input_shape=input_shape))
            model.add(LSTM(self.rnn_size))
            model.add(Dense(input_dim=self.rnn_size, units=output_shape[-1]))
        elif len(output_shape) == 2:
            # y is (num examples, max_dialogue_len, num features) so
            # all the outputs from the rnn are used to
            # calculate the loss, therefore a sequence is returned and
            # time distributed layer is used

            # the first value in input_shape is max dialogue_len,
            # it is set to None, to allow dynamic_rnn creation
            # during prediction
            model.add(Masking(mask_value=-1,
                              input_shape=(None, input_shape[1])))
            model.add(LSTM(self.rnn_size, return_sequences=True))
            model.add(TimeDistributed(Dense(units=output_shape[-1])))
        else:
            raise ValueError("Cannot construct the model because"
                             "length of output_shape = {} "
                             "should be 1 or 2."
                             "".format(len(output_shape)))

        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        logger.debug(model.summary())
        return model




agent = Agent('restaurant_domain.yml', policies=[MemoizationPolicy(max_history=3), RestaurantPolicy()])
training_data = agent.load_data('babi_stories.md')
agent.train(
            training_data,
            epochs=400,
            batch_size=100,
            validation_split=0.2
)

agent.persist('models/dialogue')

# Bölüm 2: Doğal Dil Anlama özelliğinin eklenmesi

###  Doğal Dil Anlama Modülünün Kurulumu


In [0]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config && pip install pygraphviz;
!pip install rasa_nlu[spacy];
!python -m spacy download en;

### Doğal Dil Anlama Modelinin Eğitilmesi

In [0]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer

training_data = load_data('franken_data.json')
pipeline = [{"name": "nlp_spacy"}, 
            {"name": "tokenizer_spacy"}, 
            {"name": "intent_featurizer_spacy"},  
            {"name": "intent_classifier_sklearn"},
            {"name": "ner_crf"},
            {"name": "ner_synonyms"},
           ]
trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
interpreter = trainer.train(training_data)
model_directory = trainer.persist('./projects/default/')

### Bot'un Ayağa Kaldırılması

In [0]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

### Bot Kullanıcı Arayüzü

In [0]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])


### Hikaye/Senaryo Eğitim Sonuçlarının Görüntülenmesi


In [0]:
from rasa_core.agent import Agent
from IPython.display import Image

agent = Agent.load('models/dialogue')
agent.visualize("babi_stories.md", "story_graph.png", max_history=2)
Image(filename="story_graph.png")